In [1]:
import pandas as pd
import numpy as np
import math
from nltk.stem import WordNetLemmatizer
from nltk import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('wordnet')
import re


[nltk_data] Downloading package wordnet to /home/marcela/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
df = pd.read_csv('dataset.csv')

In [33]:
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,does_this_tweet_contain_hate_speech,does_this_tweet_contain_hate_speech:confidence,_created_at,orig__golden,orig__last_judgment_at,orig__trusted_judgments,orig__unit_id,orig__unit_state,_updated_at,orig_does_this_tweet_contain_hate_speech,does_this_tweet_contain_hate_speech_gold,does_this_tweet_contain_hate_speech_gold_reason,does_this_tweet_contain_hate_speechconfidence,tweet_id,tweet_text
0,853718217,True,golden,86,NaN,The tweet uses offensive language but not hate...,0.6013,NaN,True,NaN,0.0,615561535.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,1.666196e+09,Warning: penny boards will make you a faggot
1,853718218,True,golden,92,NaN,The tweet contains hate speech,0.7227,NaN,True,NaN,0.0,615561723.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,4.295121e+08,Fuck dykes
2,853718219,True,golden,86,NaN,The tweet contains hate speech,0.5229,NaN,True,NaN,0.0,615562039.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,3.956238e+08,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandon...
3,853718220,True,golden,98,NaN,The tweet contains hate speech,0.5184,NaN,True,NaN,0.0,615562068.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,4.975147e+08,"""@jayswaggkillah: ""@JacklynAnnn: @jayswaggkill..."
4,853718221,True,golden,88,NaN,The tweet uses offensive language but not hate...,0.5185,NaN,True,NaN,0.0,615562488.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,5.889236e+08,@Zhugstubble You heard me bitch but any way I'...


In [34]:
df=df[["_unit_id","does_this_tweet_contain_hate_speech", "tweet_text"]]

In [35]:
df.head(10)


,_unit_id,does_this_tweet_contain_hate_speech,tweet_text
0,853718217,The tweet uses offensive language but not hate...,Warning: penny boards will make you a faggot
1,853718218,The tweet contains hate speech,Fuck dykes
2,853718219,The tweet contains hate speech,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandon...
3,853718220,The tweet contains hate speech,"""@jayswaggkillah: ""@JacklynAnnn: @jayswaggkill..."
4,853718221,The tweet uses offensive language but not hate...,@Zhugstubble You heard me bitch but any way I'...
5,853718222,The tweet contains hate speech,@elaynay your a dirty terrorist and your relig...
6,853718223,The tweet contains hate speech,RT @ivanrabago_: @_WhitePonyJr_ looking like f...
7,853718224,The tweet contains hate speech,Well I thought you knew actually RT @KingHorse...
8,853718225,The tweet uses offensive language but not hate...,"@Stonisnipezz I know. It was a joke, faggot."
9,853718226,The tweet uses offensive language but not hate...,I'm tired of people saying I look like my brot...


In [37]:
df.rename(columns={'_unit_id':'ID',
                    'does_this_tweet_contain_hate_speech':'Label',
                    'tweet_text':'Tweet'}, 
                 inplace=True)

In [38]:
df['Label'] = df['Label'].astype('category')

In [39]:
df['Label'] = df['Label'].cat.codes

In [40]:
df.head()

,ID,Label,Tweet
0,853718217,2,Warning: penny boards will make you a faggot
1,853718218,0,Fuck dykes
2,853718219,0,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandon...
3,853718220,0,"""@jayswaggkillah: ""@JacklynAnnn: @jayswaggkill..."
4,853718221,2,@Zhugstubble You heard me bitch but any way I'...


In [42]:
def regex(x):
    x=' '.join(re.sub("(?<=^|(?<=[^a-zA-Z0-9-\.]))@([A-Za-z0-9_]+)"," ",x).split())
    return x

In [43]:
df['Tweet'] = df['Tweet'].apply(regex)

In [44]:
df.head(10)

,ID,Label,Tweet
0,853718217,2,Warning: penny boards will make you a faggot
1,853718218,0,Fuck dykes
2,853718219,0,at least i dont look like jefree starr faggot
3,853718220,0,""" : "" : Is a fag"" jackie jealous"" Neeeee"
4,853718221,2,You heard me bitch but any way I'm back th tex...
5,853718222,0,your a dirty terrorist and your religion is a ...
6,853718223,0,RT : looking like faggots?
7,853718224,0,Well I thought you knew actually RT : Man why ...
8,853718225,2,"I know. It was a joke, faggot."
9,853718226,2,I'm tired of people saying I look like my brot...


In [190]:
lemmatizer = WordNetLemmatizer()

In [194]:
df['lemm'] = df['Tweet'].apply(lemmatizer.lemmatize)